In [ ]:
from utils_new import *

In [ ]:
pd.set_option('display.max_rows', 10)

data_dir = '/Users/erihe/OneDrive - NTNU/TDA_review/functional'

AngularRate = pd.read_pickle(data_dir + '/AngularRate')
AngularRate_Stats = pd.read_pickle(data_dir + '/AngularRate_Stats')
BorderScore = pd.read_pickle(data_dir + '/BorderScore')
BVScore = pd.read_pickle(data_dir + '/BVScore')
FilteredCells = pd.read_pickle(data_dir + '/FilteredCells')
FilteredSpikes = pd.read_pickle(data_dir + '/FilteredSpikes')
GridScore = pd.read_pickle(data_dir + '/GridScore')
GridScore_Stats = pd.read_pickle(data_dir + '/GridScore_Stats')
MECcells = pd.read_pickle(data_dir + '/MECcells')
NumberCellTypes_MEC = pd.read_pickle(data_dir + '/NumberCellTypes_MEC')
NumberCellTypes_PAS = pd.read_pickle(data_dir + '/NumberCellTypes_PAS')
OVCScore = pd.read_pickle(data_dir + '/OVCScores')
PAScells = pd.read_pickle(data_dir + '/PAScells')
Session = pd.read_pickle(data_dir + '/Session')
Tracking_Linear = pd.read_pickle(data_dir + '/Tracking_Linear')
Tracking_OpenField = pd.read_pickle(data_dir + '/Tracking_OpenField')
TrackingRaw = pd.read_pickle(data_dir + '/TrackingRaw')


# BV

In [ ]:
msesscurr = 'a936be26a1d73b28'
sess_nameof = ''
sess_names = Session[Session['metasession_name']==msesscurr]['session_name']
for sess_name0 in sess_names:
    tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
    if (((tname.lower().find('open')>-1) | (tname.lower().find('opf')>-1)) & 
        ~(tname.lower().find('circular')>-1) & 
       ~(tname.lower().find('obj')>-1)):
        sess_nameof = sess_name0  
sess_name0 = sess_nameof
cell_id = np.array(FilteredSpikes[FilteredSpikes['session_name'] == sess_name0]['cell_id'])
filtered_cellids = np.array(FilteredCells[FilteredCells['session_name']==sess_name0]['cell_id'])
filtered_bool = np.zeros(len(cell_id), dtype = bool)
for it, i in enumerate(cell_id):
    filtered_bool[it] = i in filtered_cellids
indskeep1_bu = np.where(filtered_bool)[0]
gscore =  np.array(GridScore[(GridScore['session_name'] == sess_name0) & 
                          (GridScore['signal_type'] == 'spikes') & 
                   (GridScore['map_params_id'] == 'A')]["gridscore"])
bscore  = np.array(BorderScore[((BorderScore['session_name'] == sess_name0) &
                            (BorderScore['signal_type'] == 'spikes') &
                            (BorderScore['map_params_id'] == 'A'))]['borderscore'])
bvscore =np.array(BVScore[((BVScore['session_name'] == sess_name0) &
                          (BVScore['signal_type'] == 'spikes') & 
                          (BVScore['map_params_id'] == 'A') & 
                      (BVScore['bv_field_dect_method'] == 'bvs'))]['bvs'])
ovscore = np.array(OVCScore[((OVCScore['base_session'] == sess_name0) &
                          (OVCScore['signal_type'] == 'spikes'))]['ovscore'])
hdmvl = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["mvl"])
hdmu = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["angular_mean"])
hdstd = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["angular_std"])
acorrs = np.array(GridScore[(GridScore['session_name'] == sess_name0) &
          (GridScore['signal_type'] == 'spikes') & 
          (GridScore['map_params_id'] == 'B')
         ]['acorr'])
acorrs_all = np.zeros((len(acorrs), len(acorrs[0])**2))
acorrs1 = np.zeros((len(acorrs), np.shape(acorrs[0])[0], np.shape(acorrs[0])[1]))
for i in range(len(acorrs)):
    acorrs_all[i,:] = acorrs[i].flatten()
    acorrs1[i,:, :] = acorrs[i]


Xcorr = {}
spkall = {}
xx_all = {}
yy_all = {}
tt_all = {}
hd_all = {}
for sess_name0 in sess_names:
    tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
    xx0, yy0, speed0, hd0, tt0 = [], [], [], [], []
    if (tname.lower().find('wheel')>-1):# | (tname.lower().find('circular')>-1):
        if len(Tracking_Linear[Tracking_Linear['session_name'] == sess_name0])>0:
            for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['lap_pos']:
                xx0.extend([i])
            xx0 = np.array(xx0[0])
            yy0 = np.zeros(len(xx0))
            for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['speed']:
                speed0.extend([i])
            speed0 = np.array(speed0[0])
            hd0 = np.zeros(len(xx0))
            for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['timestamps']:
                tt0.extend([i])
            tt0 = np.array(tt0[0]) 
    else:
        if len(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0])>0:
            for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['x_pos']:
                xx0.extend([i])
            xx0 = np.array(xx0[0])
            for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['y_pos']:
                yy0.extend([i])
            yy0 = np.array(yy0[0])
            for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['speed']:
                speed0.extend([i])
            speed0 = np.array(speed0[0])
            for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['head_angle']:
                hd0.extend([i])
            hd0 = np.array(hd0[0])
            for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['timestamps']:
                tt0.extend([i])
            tt0 = np.array(tt0[0])
    xx_all[sess_name0] = xx0.copy()
    yy_all[sess_name0] = yy0.copy()
#        speed_all[sess_name0] = speed0.copy()
    tt_all[sess_name0] = tt0.copy()
    hd_all[sess_name0] = hd0.copy()

speed_all = {}
for sess_name0 in sess_names:
    xspline = CubicSpline(tt_all[sess_name0], gaussian_filter1d(xx_all[sess_name0], 10))
    xx = xspline(tt_all[sess_name0])
    yspline = CubicSpline(tt_all[sess_name0], gaussian_filter1d(yy_all[sess_name0], 10))
    yy = yspline(tt_all[sess_name0])
    speed = np.sqrt(np.square(np.diff(xx)) + np.square(np.diff(yy)))
    speed = np.concatenate(([speed[0]],speed)) # m/10ms
    speed_all[sess_name0] = speed*100


scores = (('acorr2d', acorrs1),
     ('hdmvl', hdmvl),
       ('bscore', bscore),
       ('bvscore', bvscore),
     )

spk0_all = {}
for sess_name0 in sess_names:
    spikes0 = []
    for it, i in enumerate(FilteredSpikes[FilteredSpikes['session_name'] == sess_name0]['filtered_spikes']):
        spikes0.append(i)
        it += 1
    spk0_all[sess_name0] = np.array(spikes0).T       

for sess_name0 in xx_all:
    if len(xx_all[sess_name0]) == 0:
        xx_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))
        yy_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))
        speed_all[sess_name0] = np.ones(len(spk0_all[sess_name0].copy()))
        tt_all[sess_name0] = np.arange(len(spk0_all[sess_name0].copy()))
        hd_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))

for sig0 in [2]:
    for lenc in [10]:
        for sess_name0 in sess_names:
            lenspk, num_neurons = np.shape(spk0_all[sess_name0])           
            spikes1 = preprocessing.minmax_scale(spk0_all[sess_name0], axis = 0)
            if sig0>0:
                spikes1 = np.sqrt(gaussian_filter1d(spikes1, axis = 0, sigma = sig0))
            else:
                spikes1 = np.sqrt(spikes)                                            
#                spikes1 = preprocessing.minmax_scale(spikes1, axis = 0)
            Xcorr[sess_name0] = cross_corr_dist(spikes1, lencorr = lenc)
            print('sig, lenc', sig0, lenc)
            Xcorr[sess_name0][np.isnan(Xcorr[sess_name0])] = 0    


In [ ]:
dd1 =np.zeros((len(indskeep1_bu), len(indskeep1_bu)))
for i, sess in enumerate(Xcorr):
    xx1 = Xcorr[sess].copy()
    xx1 = xx1[indskeep1_bu,:]
    xx1 = xx1[:,indskeep1_bu]
    xx1 = np.square(xx1)
    np.fill_diagonal(xx1,0)
    xx1 = squareform(pdist(xx1, 'correlation'))
    dd1 += xx1
dd1 /= len(Xcorr)

tested = [-1]
thr = 0.91
ind1 = get_ind(dd1,thr, linkage = 'average', bPlot = False)
bin_ind = np.bincount(ind1)
tested.extend([np.max(bin_ind)])
numneuronsind = np.flip(np.argsort(bin_ind))
print('num: ', bin_ind[numneuronsind[:10]])
print('ind: ', numneuronsind[:10])
print('')

fig = plt.figure(dpi = 300)
d2 = dd1.copy()
d2 = d2[np.argsort(ind1), :]
d2 = d2[:,np.argsort(ind1)]    

plt.imshow(d2, 
           vmin = np.percentile(d2.flatten(),5),
           vmax = np.percentile(d2.flatten(),95),
          )
plt.axis('off')
bin_ind = np.bincount(ind1)
numneuronsind = np.flip(np.argsort(bin_ind))
print('num: ', bin_ind[numneuronsind[:10]])
print('ind: ', numneuronsind[:10])
plt.savefig('Figures/BV_corrmat.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/BV_corrmat.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

data = []
data_names = []
for i in range(len(d2)):
    data.append(pd.Series(d2[:,i]))
    data_names.extend(['col_' + str(i)])
df = pd.concat(data, ignore_index=True, axis=1)            
df.columns = data_names
df.to_excel("Source_data/ExtFig1b_BV.xlsx", sheet_name='crossmat')  


In [ ]:
sig = 2
numinds = np.bincount(ind1)
rel_inds = np.where((numinds>=10) & (numinds<=500))[0]                             
for i in rel_inds:
    mod_ind1s = indskeep1_bu[(ind1==i) ]
    if len(mod_ind1s)>=2:
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        sspk2 = spk0_all[sess_nameof][:,mod_ind1s]
        sspk2 = preprocessing.minmax_scale(sspk2)
        sspk2 = np.sqrt(sspk2)
        sspk2 = gaussian_filter1d(sspk2,sigma = sig, axis = 0)
        movetimes00 = np.arange(0,len(sspk2),1)
        xx1 = xx_all[sess_nameof]
        yy1 = yy_all[sess_nameof]

        times1 = np.round(np.linspace(0, len(xx1)-1, len(sspk2))).astype(int)
        scores_cluster(sspk2,scores,  mod_ind1s,xx1[times1],yy1[times1], spk2 = [], 
                       num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
        plt.show()

In [ ]:
n_all = [600]
eps_all = [0.2, ]
dim_all = [4,]
hdmetric = np.zeros((len(sess_names), len(eps_all), len(dim_all), len(n_all)))
sig = 2
numinds = np.bincount(ind1)
minnum = 67
maxnum = 300
rel_inds = np.where((numinds>=minnum) & (numinds<=maxnum))[0]                             
for i in rel_inds:
    mod_ind1s = indskeep1_bu[(ind1==i) ]
    if len(mod_ind1s)>=2:
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        sspk2 = spk0_all[sess_nameof][:,mod_ind1s]
        sspk2 = preprocessing.minmax_scale(sspk2)
        sspk2 = np.sqrt(sspk2)
        sspk2 = gaussian_filter1d(sspk2,sigma = sig, axis = 0)
        movetimes00 = np.arange(0,len(sspk2),1)
        xx1 = xx_all[sess_nameof]
        yy1 = yy_all[sess_nameof]
        hd1 = hd_all[sess_nameof]

        times1 = np.round(np.linspace(0, len(xx1)-1, len(sspk2))).astype(int)
        bTime = True
        numtimes = 1
        bMean = True    
        k = 1000
        maxdim = 1   
        metric = 'cosine'
        for sp in [50,]:
            for pthr in [0]:
                for epsit, eps in enumerate(eps_all):
                    for dimit, dim in enumerate(dim_all):
                        for sig in [2,]:
                            for sessit, sess_name0 in enumerate(sess_names[:]):
                                print(sess_name0)
                                tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
                                print(tname)
                                speed0 = speed_all[sess_name0]
                                hd0 = hd_all[sess_name0]
                                tt0 = tt_all[sess_name0]
                                xx0 = xx_all[sess_name0]
                                yy0 = yy_all[sess_name0]
                                sspk1 = spk0_all[sess_name0][:,mod_ind1s]
                                sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
                                sspk1 = gaussian_filter1d(sspk1,sigma = sig, axis = 0)
                                sspk1 = np.sqrt(sspk1)                                
                                if bTime:
                                    movetimes0 =np.arange(0, len(sspk1),numtimes)
                                    print('movetimes0', len(movetimes0))            
                                times0 = np.round(np.linspace(0, len(xx0)-1, len(sspk1))).astype(int)
                                if sp>0:
                                    movetimes0 = movetimes0[np.where(speed0[times0][movetimes0] >=sp)[0]]
                                    print('movetimes0', len(movetimes0))
                                print('movetimes0', len(movetimes0))            
                                movetimes0 = movetimes0[np.where(np.sum(sspk1[movetimes0,:],1)>=0)[0]]
                                print('movetimes0', len(movetimes0))            
                                if bMean:
                                    movetimes0 = movetimes0[np.where(np.sum(sspk1[movetimes0,:],1)>=
                                                                     np.percentile(np.sum(sspk1[movetimes0,:],1), pthr))[0]]
                                    print('movetimes0', len(movetimes0))

                                sspk1 = sspk1[movetimes0,:]
                                times = times0[movetimes0]
                                indstemp = []
                                n_points = 2000

                                spk1 = preprocessing.scale(sspk1,axis = 0)
                                dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
                                dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
                                startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
                                movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, epsilon = eps, 
                                    startindex = startindex)
                                if n_points > len(movetimes1):
                                    n_points = len(movetimes1)
                                indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                                                   n_points, 1, metric = 'cosine')[0]
                                indstemp = movetimes1[indstemp]
                                for nit, n_points in enumerate(n_all):
                                    print('eps', eps, dim, n_points)
                                    if (len(indstemp)>0) & (n_points > len(indstemp)):
                                        print(n_points, len(indstemp))
                                        continue
                                    indstemp = indstemp[:n_points]
                                    dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

                                    d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
                                    thresh = np.max(d[~np.isinf(d)])    
                                    persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
                                    print('movetimes1', len(movetimes1))
                                    n_points = len(indstemp)

                                    if (len(movetimes1) < n_points) & (len(movetimes1)>0):
                                        print('movetimes1 less than n_points')
                                        indstemp = []
                                        continue
                                    try:
                                        coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
                                    except:
                                        continue
                                    times  = movetimes0[indstemp].copy()
                                    dgms = persistence['dgms']



                                    fig = plot_barcode(dgms)
                                    fig,ax = plt.subplots(1,1)
                                    ax.plot(var_exp[:15], lw = 2.5)
                                    ax.spines['top'].set_visible(False)
                                    ax.spines['right'].set_visible(False)
                                    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
                                    for i in range(len(coords_ds)):
                                        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
                                    ax[2].scatter(*coords_ds[:2,:], s = 100)
                                    for i in range(3):
                                        ax[i].set_aspect(1/ax[i].get_data_ratio())

                                    fig, ax = plt.subplots(1,3, figsize = (10,5), dpi = 120)
                                    for i in range(len(coords_ds_consistent)):
                                        ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
                                    ax[2].scatter(*coords_ds_consistent[:2,:], s = 100)
                                    for i in range(3):
                                        ax[i].set_aspect(1/ax[i].get_data_ratio())


                                    coords_mod1 = get_coords_all(sspk1, 
                                                                 coords_ds,
                                                                 np.arange(len(sspk1)),                             
                                                                 indstemp,
                                                                 dim = dim, 
#                                                                         spk2 = sspk2,
                                                                 bPCA = True,
                                                                 bPred = False)
                                    times = times0[movetimes0]
                                    coords_mod1 = coords_mod1%(2*np.pi)
                                    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
                                    for c in [0,1,]:
                                        mtot, __, __, circ  = binned_statistic_2d(xx0[times],
                                                                                  yy0[times],
                                                                                  coords_mod1[:,c], 
                                                                                  statistic = circmean, 
                                                                                  bins = 50,
                                                                                  expand_binnumbers = True)

                                        nans = np.isnan(mtot)
                                        sintot = np.sin(mtot)
                                        costot = np.cos(mtot)
                                        sintot[nans] = np.mean(sintot[~nans])
                                        costot[nans] = np.mean(costot[~nans])
                                        sintot = gaussian_filter(sintot,sig)
                                        costot = gaussian_filter(costot,sig)
                                        mtot = np.cos(np.arctan2(sintot, costot))
                                        plt.viridis()
                                        mtot[nans] = np.nan
                                        axs1[c].imshow(mtot)
                                        axs1[c].axis('off')
                                        axs1[c].set_aspect(1/axs1[c].get_data_ratio())
                                    plt.show()


In [ ]:

diagrams_roll = {}
count = -1
for i in range(0,101):
    if i == 0:
        f = np.load('Obenhaus/dgms' + str(0) + '.npz', allow_pickle = True)
        dgms = f['dgms'][()][0]
        f.close()
        continue
    try :
        f = np.load('Obenhaus/dgms' + str(i) + '.npz', allow_pickle = True)
        dgmstmp = f['dgms'][()]
        f.close()
        count += 1
        diagrams_roll[count] = dgmstmp[0]
    except:
        continue
xmax = 0.5

plot_barcode(dgms, diagrams_roll = diagrams_roll, percshuf = 99, dpi = 300, 
             SaveSourceDataName = 'BV_barcode')
plt.savefig('Figures/BV_barcode.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/BV_barcode.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

In [ ]:
sspk1 = spk0_all[sess_name0][:,mod_ind1s]
sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
sspk1 = gaussian_filter1d(sspk1,sigma = sig, axis = 0)
sspk1 = np.sqrt(sspk1)                                

coords_mod1 = get_coords_all(sspk1, 
                         coords_ds,
                         movetimes0,
                         indstemp,
                         dim = dim, 
                         bPCA = True,
                         bPred = False)

plt.hsv()
fig, ax = plt.subplots(1,1,dpi = 300,figsize = (5,5))
ax.axis('off')
ax.scatter(xx0[times0], yy0[times0], c = coords_mod1[:,0])


data = []
data_names = []
data.append(pd.Series(xx0[times0]))
data_names.extend(['xpos'])
data.append(pd.Series(yy0[times0]))
data_names.extend(['ypos'])

data.append(pd.Series(coords_mod1[:,0]))
data_names.extend(['decoding'])

df = pd.concat(data, ignore_index=True, axis=1)            
df.columns = data_names
df.to_excel('Source_data/ExtFig1b_BV_spatial_decoding.xlsx', sheet_name='BV_spatial_decoding')  
plt.savefig('Figures/BV_spatial_decoding.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/BV_spatial_decoding.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)



In [ ]:
sspk1 = spk0_all[sess_name0][:,mod_ind1s]
num_neurons = len(sspk1[0,:])
hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    spktmp = sspk1[:,i].copy()
    mtot, __,  circ  = binned_statistic(coords_mod1[:,0], spktmp,
                                              statistic = 'mean', 
                                              bins = 30)
    mu = np.mean(spktmp)
    hd_info[i] = information_score_1d(mtot, circ-1, mu)


In [ ]:
coords_mod1 = (coords_mod1 + np.pi/2)%(2*np.pi)
coords_mod1 = 2*np.pi-coords_mod1 

In [ ]:
from matplotlib import gridspec
torsort = np.flip(np.argsort(hd_info))
bv_cells = np.array(torsort)[[4, 3, 1, 0 ]]
numfigs = 2
numw = 2
num_neurons = len(bv_cells)
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 300)
nw = 0
numbins1 = 30
for nn, n in enumerate(bv_cells):
    nnn = nn%numh

    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .4)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .2)
        nw += 1
    xnum = 0

    ax = plt.subplot(gs2[nnn,xnum]) 
    xnum += 1
    mtot, __, __, circ  = binned_statistic_2d(xx[times0],
                                          yy[times0],
                                          sspk1[:,n], 
                                          statistic = 'mean', 
                                          bins = 30,
                                          expand_binnumbers = True)

    nans = np.isnan(mtot)
    mtot[nans] = np.mean(mtot[~nans])
    mtot = gaussian_filter(mtot, 1)
    plt.viridis()
    vals = np.unique(mtot)
    mtot[nans] = np.nan
    
    maxtot = np.sort(mtot.flatten())
    mintot = maxtot[int(0.025*len(maxtot))]
    maxtot = maxtot[int(0.975*len(maxtot))]
    ax.imshow(mtot, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = mintot, vmax = maxtot)
    ax.set_xticks([])
    ax.set_yticks([])

    ax = plt.subplot(gs2[nnn,xnum], projection = 'polar')
    xnum += 1
    
    bins = np.linspace(0,2*np.pi,31)

    dec = binned_statistic(coords_mod1[:,0], sspk1[:,n], statistic = 'mean', bins=bins, range=None)[0]
    dec[np.isnan(dec)] = 0
    dec /= np.max(dec)
    dec = gaussian_filter1d(dec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([dec[-1]], dec)), 
            lw = 4, ls = '-', alpha = 0.9, c = [0.,0.5,0.5], label = 'Dec OF')

    ax.set_rticks([0.33, 0.66,1]) 
    ax.set_yticklabels('')
    ax.set_xticklabels('')
    ax.set_rlim([0,1.1])

    for i in range(len(mtot)):
        data.append(pd.Series(mtot[:,i]))
        data_names.extend(['spatial_' + str(i)])

    data.append(pd.Series(dec))
    data_names.extend(['polar_tuning'])

    df = pd.concat(data, ignore_index=True, axis=1)            
    df.columns = data_names    
    df.to_excel('Source_data/Obenhaus_BV_single_tuning' + str(nn) + '.xlsx', sheet_name='Obenhaus_BV_' + str(nn))  
plt.savefig('Figures/Obenhaus_BV_single_tuning.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/Obenhaus_BV_single_tuning.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

